# P2: 20 Questions
#### Group 2: Tyler Henson, Tyler Husemann, Brandon Lowery, and Ean Vandergraaf
#### 523 Artificial Intelligence

## 0. Background Information



For our 20-Questions model, we decided to assign each of the 1850 nouns in the dataset to one of 20 categories. These categories represent broad semantic groups such as Vehicles, Tools, Fruits, Wild Mammals, Buildings & Structures, etc. Our goal was to help the 20-Questions model narrow down possibilities more efficiently during gameplay. Our theory was that we could identify a noun's category within 4-5 questions. Then the model would be able to ask property rating quesitons to precisely pin down the user's noun.

The code provided below is our final model. It utilizes 2 external files which were compiled using the `noun_categorizer.py` and `Generate_Ratings.ipynb` files. 



## 1. 20 Questions AI

In [93]:
import json

# Load category tree
with open('categories_tree.json', 'r') as f:
    tree = json.load(f)

# Load noun property data
with open("noun_property_ratings.jsonl", "r") as file:
    raw_data = [json.loads(line) for line in file]

questions = tree["Questions"]
categories = tree["Categories"]

# Greedy algorithm to select best question
def best_question(noun_data, remaining_nouns, asked_properties, properties, property_types):
    best_property = None
    best_split_score = float("inf")
    best_threshold = None
    best_type = None
    best_reference = None

    for prop in properties:
        if prop in asked_properties:
            continue

        values = [(noun, noun_data[noun][prop]) for noun in remaining_nouns if prop in noun_data[noun]]

        if property_types[prop] == "boolean":
            yes = [v for _, v in values if v == 1]
            no = [v for _, v in values if v == 0]
            split_score = abs(len(yes) - len(no))

            if split_score < best_split_score:
                best_property = prop
                best_split_score = split_score
                best_type = "binary"

        elif property_types[prop] == "scale":
            sorted_values = sorted(values, key=lambda x: x[1])
            for i in range(1, len(sorted_values)):
                threshold = sorted_values[i][1]
                yes = [v for _, v in values if v > threshold]
                no = [v for _, v in values if v <= threshold]
                split_score = abs(len(yes) - len(no))

                if 0 < len(yes) < len(values) and split_score < best_split_score:
                    best_property = prop
                    best_split_score = split_score
                    best_threshold = threshold
                    best_type = "continuous"
                    best_reference = sorted_values[i][0]

    return best_property, best_type, best_threshold, best_reference

def run_ai(questions):
    remaining_questions = 20
    
    # 1. Traverse decision tree to select category

    print('Enter 1 for YES, 0 for NO')
    node = questions
    while not isinstance(node, str):
        question = node["Question"]
        answer = input(question + " (0/1): ").strip().lower()
        print(f"{remaining_questions}. {question}: {"YES" if answer == "1" else "NO"}")
        if answer == '1':
            node = node["YES"]
            remaining_questions -= 1
        elif answer == '0':
            node = node["NO"]
            remaining_questions -= 1
        else:
            print("Invalid input. Please enter 1 for YES or 0 for NO.")
            continue

    selected_category = node
    # print(f"\nSelected Category: {selected_category}")
    # print(f"# of Options: {len(categories[selected_category])}")

    # 2. Use Greedy Algorithm to picm the best questions to ask

    # Filter data by selected category
    filtered_data = [entry for entry in raw_data if entry["category"] == selected_category]

    # Organize data: noun -> {property: rating}
    noun_data = {}
    property_types = {}
    for entry in filtered_data:
        noun = entry["noun"]
        prop = entry["property"]
        rating = entry["rating"]
        ptype = entry["property_type"]

        if noun not in noun_data:
            noun_data[noun] = {}
        noun_data[noun][prop] = rating
        property_types[prop] = ptype

    # Extract all properties
    properties = set(property_types.keys())

    remaining_nouns = set(noun_data.keys())
    asked_properties = set()
    question_count = 0

    fallback_answer = "Football" ## in case there is a problem with our logic, will get the next closest answer
    while question_count < 19 and len(remaining_nouns) > 1:
        fallback_answer = list(remaining_nouns)[0]
        best_prop, prop_type, threshold, reference_noun = best_question(noun_data, remaining_nouns, asked_properties, properties, property_types)

        if best_prop is None:
            break
        
        question = ""
        if prop_type == "binary":
            question = best_prop
            answer = int(input(f"{question.capitalize()} (0/1): "))

        elif prop_type == "continuous":
            if reference_noun:
                question = "Is its " + best_prop + " greater than the " + best_prop + " of a " + reference_noun + "?"
                answer = int(input(f"{question.capitalize()} (0/1):"))
            else:
                question = "Is its " + best_prop + " greater than " + threshold + "?"
                answer = int(input(f"{question.capitalize()} (0/1):"))
        print(f"{remaining_questions}. {question.capitalize()}: {"YES" if answer == 1 else "NO"}")
        remaining_questions -= 1

        # Filter remaining nouns
        if prop_type == "binary":
            remaining_nouns = {noun for noun in remaining_nouns if noun_data[noun].get(best_prop) == (1 if answer == 1 else 0)}
        elif prop_type == "continuous":
            if answer == 1:
                remaining_nouns = {noun for noun in remaining_nouns if noun_data[noun].get(best_prop, 0) > threshold}
            else:
                remaining_nouns = {noun for noun in remaining_nouns if noun_data[noun].get(best_prop, 0) <= threshold}

        asked_properties.add(best_prop)
        question_count += 1
        # print(remaining_nouns)
        # print(f"{len(remaining_nouns)} nouns remaining.")

    guess = list(remaining_nouns)[0] if remaining_nouns else fallback_answer
    print(f"\nFinal Guess: Are you thinking of '{guess}'?")
    return guess



## 2. Reflection

### Overall Approach and Design Process

#### Noun Categorization Method

The first part of our 20-Questions model deals with identifying the category of the noun. These categories allow our model to ask smarter questions instead of guessing randomly. These categories represent broad types of things (like Vehicles, Tools, Foods, Animals, etc.). Our group identified 20 distinct categories and formed a binary tree which would lead the model to the correct category (`category_tree.json`).

To assign categories, we used the **Phi-3 Mini language model** through **Ollama**. We gave the model:
- The list of allowed categories
- A brief description of each category
- One noun at a time to classify

The `noun_categorizer.py` file was used to categorize all our nouns. The Mini model suggested which category each noun belonged to, and we reviewed the results to fix unclear or incorrect assignments. Some nouns were harder to categorize (brand names, fictional characters, slang, etc.), so we refined prompts and re-ran only those. After final adjustments, we combined everything into a single file (`categorized_nouns.json`) containing all nouns and assigned categories. 

Once the categories were set, the game traversed the tree to guide its questioning strategy. This categorization step was the foundation for the rest of the system, as it directly influenced how efficiently the model could narrow down answers.

#### Property Rating Method

Once the model has identified a category, it's next move was to start asking rank based questions. These consisted of questions such as "Is its weight greater than the weight of a trash can?" This would allow the model to significantly narrow down the list of nouns per category. In order to do this our group prompted the Mini model with all the nouns, categories, and properties. This process can be found in `Generate_Ratings_final.ipynb`.The output, `noun_property_ratings.json`, consisted of all variations of nouns, properties, property types (boolean or scale), and it's noun-property ranking. 

Once the properties were set, our 20-Questions model used a greedy algorithm to select the next best question. Depending on the answer to that question, our model would decide were to pivot next based on the remaining nouns. The model will continue to ask these ranking based questions until there was 1 option left.


### Overall Performance

After testing the 20 Questions game several times, the results were mixed but generally reasonable. When the noun had a clear and common category, the model tended to guess correctly or very close. For example, words like Tiger, Hamburger, and Ice Cream were guessed correctly. Words with slightly broader or overlapping meanings were sometimes guessed as something similar instead of exact. For example, during testing, trying to guess Bulldog resulted in the model suggesting Poodle. This shows that the model understands the general type of animal, but its narrowing logic isn't always precise.

Some nouns still challenge the model, especially:
- Fictional characters
- Objects with multiple meanings
- Nouns that fit more than one possible category

So the model usually narrowed the search down to a similar group rather than something completely unrelated, which indicates that the category system is working as intended. With more tuning in how questions are selected and how properties are weighted, the model’s accuracy could likely improve further.


### Example Games

In [72]:
run_ai(tree["Questions"]) ## tiger

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: YES
19. Is it currently alive (not a food or processed product)?: YES
18. Is it an animal?: YES
17. Is it a mammal?: YES
16. Is it a domesticated pet or farm animal?: NO
15. Is its speed greater than the speed of a hedgehog?: YES
14. Is it a predator?: YES
13. Is its size greater than the size of a leopard?: YES
12. Is it african?: NO
11. Is it social?: NO
10. Does it hibernate?: NO
9. Is it endangered?: YES
8. Is it nocturnal?: NO
7. Is it herbivore?: NO

Final Guess: Are you thinking of 'tiger'?


'tiger'

In [73]:
run_ai(tree["Questions"]) ## airplane

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: NO
19. Is it a vehicle or mode of transportation?: YES
18. Does it have an engine or motor?: YES
17. Does it fly?: YES
16. Is its cost greater than the cost of a thruster?: YES
15. Does it have wings?: YES
14. Is its size greater than the size of a airplane?: YES
13. Does it need a runway?: YES

Final Guess: Are you thinking of 'jet'?


'jet'

In [75]:
run_ai(tree["Questions"]) ## hamburger

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: YES
19. Is it currently alive (not a food or processed product)?: NO
18. Is it a prepared dish or meal (cooked/assembled)?: YES
17. Is its healthy greater than the healthy of a mojito?: NO
16. Is its complexity greater than the complexity of a cake?: NO
15. Is it dessert?: NO
14. Is it cuisine-specific?: NO
13. Does it have meat?: YES

Final Guess: Are you thinking of 'hamburger'?


'hamburger'

In [79]:
run_ai(tree["Questions"]) ## dandelion

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: YES
19. Is it currently alive (not a food or processed product)?: YES
18. Is it an animal?: NO
17. Is it a tree?: NO
16. Is it colorful: YES
15. Is its height greater than the height of a lilac?: NO
14. Is it annual?: NO
13. Is it fragrant?: YES

Final Guess: Are you thinking of 'lilac'?


'lilac'

In [80]:
run_ai(tree["Questions"]) ## ice cream

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: NO
Invalid input. Please enter 1 for YES or 0 for NO.
20. Is it a living thing or was it ever alive?: YES
19. Is it currently alive (not a food or processed product)?: NO
18. Is it a prepared dish or meal (cooked/assembled)?: YES
17. Is its healthy greater than the healthy of a mojito?: NO
16. Is its complexity greater than the complexity of a cake?: NO
15. Is it dessert?: YES
14. Does it have dairy?: YES

Final Guess: Are you thinking of 'ice cream'?


'ice cream'

In [85]:
run_ai(tree["Questions"]) ## shaquille o'neal

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: YES
19. Is it currently alive (not a food or processed product)?: YES
18. Is it an animal?: YES
17. Is it a mammal?: YES
16. Is it a domesticated pet or farm animal?: NO
15. Is its speed greater than the speed of a hedgehog?: YES
14. Is it a predator?: NO
13. Is it herbivore?: NO
12. Is it social?: YES
11. Is it african?: NO
10. Does it hibernate?: NO
9. Is it endangered?: NO
8. Is it nocturnal?: NO

Final Guess: Are you thinking of 'ferret'?


'ferret'

In [94]:
run_ai(tree["Questions"]) ## eiffel tower

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: NO
19. Is it a vehicle or mode of transportation?: NO
18. Is it a natural formation or geological feature?: NO
17. Is it primarily infrastructure for transportation (roads, bridges, airports, etc.)?: NO
16. Is it a building or large structure?: YES
15. Is its size greater than the size of a yard?: YES
14. Is it residential?: NO
13. Is it a landmark?: YES
12. Is it urban?: YES
11. Is it tall?: YES

Final Guess: Are you thinking of 'eiffel tower'?


'eiffel tower'

In [95]:
run_ai(tree["Questions"]) ## guitar

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: NO
19. Is it a vehicle or mode of transportation?: NO
18. Is it a natural formation or geological feature?: NO
17. Is it primarily infrastructure for transportation (roads, bridges, airports, etc.)?: NO
16. Is it a building or large structure?: NO
15. Is it a physical object you can hold in your hands?: YES
14. Is its sharpness greater than the sharpness of a ladder?: NO
13. Is its weight greater than the weight of a trash can?: NO
12. Is it a tool?: NO
11. Is it a household item?: YES
10. Is it electricity powered?: YES
9. Is it metal?: YES

Final Guess: Are you thinking of 'desk lamp'?


'desk lamp'

In [96]:
run_ai(tree["Questions"]) ## bulldog

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: YES
19. Is it currently alive (not a food or processed product)?: YES
18. Is it an animal?: YES
17. Is it a mammal?: YES
16. Is it a domesticated pet or farm animal?: YES
15. Is its social greater than the social of a buffalo?: YES
14. Is its maintenance greater than the maintenance of a cavalier king charles spaniel?: NO
13. Is its trainable greater than the trainable of a cavalier king charles spaniel?: YES
12. Is it an indoor pet?: YES
11. Is its lifespan greater than the lifespan of a miniature schnauzer?: YES
10. Does it have fur?: YES
9. Is it for work?: NO
8. Is it a pet?: YES
7. Is it a carnivore?: YES

Final Guess: Are you thinking of 'poodle'?


'poodle'

In [97]:
run_ai(tree["Questions"]) ## superman

Enter 1 for YES, 0 for NO
20. Is it a living thing or was it ever alive?: YES
19. Is it currently alive (not a food or processed product)?: YES
18. Is it an animal?: YES
17. Is it a mammal?: YES
16. Is it a domesticated pet or farm animal?: NO
15. Is its speed greater than the speed of a hedgehog?: YES
14. Is it a predator?: NO
13. Is it herbivore?: NO
12. Is it social?: YES
11. Is it african?: NO
10. Does it hibernate?: NO
9. Is it endangered?: NO
8. Is it nocturnal?: NO

Final Guess: Are you thinking of 'ferret'?


'ferret'

### Performance Summary

After running our 20 Questions bot 10 times we had some varying responses.

<table>
    <th>Noun</th>
    <th>Guess</th>
    <th># of Questions</th>
    <th>Correctness</th>
    <tr>
        <td>Tiger</td>
        <td>Tiger</td>
        <td>14</td>
        <td><span style="color: green; font-weight: bold;">Correct</span></td>
    </tr>
    <tr>
        <td>Airplane</td>
        <td>Jet</td>
        <td>8</td>
        <td><span style="color: yellow;">Nearly Close</span></td>
    </tr>
    <tr>
        <td>Hamburger</td>
        <td>Hamburger</td>
        <td>8</td>
        <td><span style="color: green; font-weight: bold;">Correct</span></td>
    </tr>
    <tr>
        <td>Dandelion</td>
        <td>Lilac</td>
        <td>8</td>
        <td><span style="color: yellow;">Nearly Close</span></td>
    </tr>
    <tr>
        <td>Ice Cream</td>
        <td>Ice Cream</td>
        <td>9</td>
        <td><span style="color: green; font-weight: bold;">Correct</span></td>
    </tr>
    <tr>
        <td>Shaquille O'neal</td>
        <td>Ferret</td>
        <td>13</td>
        <td><span style="color: red; font-weight: bold;">Wrong</span></td>
    </tr>
    <tr>
        <td>Eiffel Tower</td>
        <td>Eiffel Tower</td>
        <td>10</td>
        <td><span style="color: green; font-weight: bold;">Correct</span></td>
    </tr>
    <tr>
        <td>Guitar</td>
        <td>Desk Lamp</td>
        <td>12</td>
        <td><span style="color: red; font-weight: bold;">Wrong</span></td>
    </tr>
    <tr>
        <td>Bulldog</td>
        <td>Poodle</td>
        <td>14</td>
        <td><span style="color: yellow;">Nearly Close</span></td>
    </tr>
    <tr>
        <td>Superman</td>
        <td>Ferret</td>
        <td>13</td>
        <td><span style="color: red; font-weight: bold;">Wrong</span></td>
    </tr>
</table>

### Future Work and Limitations

A portion of nouns were difficult to classify:
- Brand names  
- Fictional characters  
- Very technical or niche terms  
- Ambiguous or slang words  

Some of these were initially placed in “Other” or left uncategorized. We re-ran those nouns separately with refined prompting to reduce the number of unclear results. A small number may still remain ambiguous.

Overall, the game worked well once we had a full categorized noun list. It was able to narrow things down quickly in most runs. The times it struggled were usually with nouns that are a bit unclear, like fictional characters, celebrities, or things that don’t fit cleanly into one category. In those cases, the model sometimes chose something “close” but not correct.

If we continued this project, we would focus on:
- Breaking some categories into smaller or more specific groups (especially for people/characters).
- Giving the model short definitions or context for tricky nouns to help it understand them better.
- Letting the game learn from previous wrong guesses to improve over time.
- Fine tuning the initial question asked. 

These improvements would help reduce the number of “almost” or incorrect guesses and make the game more consistent overall.

